<a href="https://colab.research.google.com/github/Yan-Weijia/CompLing/blob/master/HW0401.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

извлекать сущность "Игра для приставки" или "Аксессуар для приставки"

Напишите правила в yargy для извлечения одной из выбранных сущностей. Начните с самого простого правила и понемногу добавляйте условий. 
Остановитесь, когда на датасете в ~9к текстов у вас получится извлекать хотя бы 500 сущностей (не обязательно уникальных).


In [0]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)

In [0]:
!wget https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/pristavki.csv

In [0]:
data = pd.read_csv('pristavki.csv.1', header=None, names=['text'])

In [28]:
data.head(20)

,text
0,"Состояние: Отличное для возраста приставки./\nИгры отдам вместе с приставкой./\nОстальное по телефону, либо при осмотре."
1,"+5 игар фифа15,16 мартол комбат 9, гта5,асассин 4, и три диска Фифв 13 гта 4 видмак3"
2,"Продам в отличном состоянии , в комплекте проводной и беспроводной геймпады (зарядное устройство для беспроводного), гарнитура , два диска . Вообщем полный комплект, но без Kinect."
3,"Продам PS3, 2 беспроводных джойстика в комплекте. Коллекция игр будет естественно бонусом) на фото не хватает игры Mortal combat vs D.C. Джойстики не залипают,можете проверить при покупке. Обмен не интересен, так как подарили Ps4) небольшой торг )"
4,"Продаю в связи с ненадобностью.Купила для мужа ,играть не стал.Состояние идеальное .Любые проверки.Предложение идеально до 21 апреля"
5,"Код не активировал,обмен"
6,Возможен обмен
7,"Xbox в отличном состоянии, 2 диска с играми /\n/\nПоддержка носителей: CD, DVD, флэшка./\n/\nWi-Fi (802.11)/\nBluetooth. Цвет: черный"
8,"Обмен интересен на FIFA16,17 или на GTA5"
9,"Джойстик в стиле PS1 для игровых приставок Dendy. Цвет — белый./\n /\nНаш магазин находятся по адресу: /\n*** г.Зеленодольск, Татарстан 10 2 этаж, График работы магазина: с 10:00-20:00, Без выходных/\n /\n❀❀❀❀❀Почему наши покупатели всегда довольны❀❀❀❀❀/\n✔ЗОЛОТАЯ МЕДАЛЬ от авито в графе продавец!/\n✔нас более 50 магазинов по всей России, что позволяет нам держать самые низкие цены за счет прямых поставок с фабрик/\n✔отличное качество и огромный ассортимент!/\n✔постоянные супер-выгодные для покупателей акции/\n✔подарки за покупки от 2000 руб/\n✔заводская гарантия 6 месяцев/\n✔все товары в наличии (если что-то закончилось - то, как правило, находится в пути)/\n✔возле магазинов имеются парковки!/\n✔чтобы нас не потерять - добавляйтесь в группу ""Вконтакте"" и Инстаграм ""Мир удивительных товаров""!/\n /\n✔Работаем ЕЖЕДНЕВНО!/\n /\n☎ ЗВОНИТЕ ПРЯМО СЕЙЧАС и получите консультацию!/\n______________________/\nБудем рады видеть Вас в числе наших довольных покупателей!/\nЕЩЕ БОЛЬШЕ НУЖНЫХ ВАМ ТОВАРОВ НА ФОТОГРАФИЯХ НИЖЕ/\n↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓"


In [0]:
!pip install yargy

In [0]:
from yargy import Parser, rule, or_
from yargy.predicates import in_, in_caseless
from yargy.tokenizer import MorphTokenizer
from yargy.pipelines import morph_pipeline, caseless_pipeline
from yargy.interpretation import fact
from IPython.display import display

In [0]:
Game = fact(
    'Game',
    ['name', 'version']
)

In [0]:
FIFA = rule(
    morph_pipeline(['FIFA', 'fifa', "фифа", 'ФИФА']).interpretation(Game.name), 
    morph_pipeline(['15', '16', '17', '18', '19', '2015', '2016', '2017', '2018', '2019']).interpretation(Game.version))

In [0]:
GTA = rule(
    morph_pipeline(['GTA', 'gta', "гта", 'ГТА']).interpretation(Game.name), 
    morph_pipeline(['5', '4', 'IV', 'V', 'III', '3']).interpretation(Game.version)) 

In [0]:
Witcher = rule(
    morph_pipeline(['The Witcher', 'The witcher', 'the witcher', 'THE WITCHER', 'видмак', 'ВИДМАК', 'Видмак']).interpretation(Game.name), 
    morph_pipeline(['1', '2', '3', 'one', 'two', 'three']).interpretation(Game.version)) 

In [0]:
Red_dead = rule(
    morph_pipeline(['Red Dead Redemption', 'Red Dead', 'ред дед редемпшн', 'ред дед']).interpretation(Game.name), 
    morph_pipeline(['1', '2', 'one', 'two']).interpretation(Game.version)) 

In [0]:
SSF = rule(
    morph_pipeline(['Super Street Fighter', 'Street fighter', "SSF", 'супер стрит файтер']).interpretation(Game.name), 
    morph_pipeline(['I', '1', 'II', '2', 'III', '3', 'IV', '4', 'V', '5']).interpretation(Game.version))

In [0]:
GAME = or_(FIFA, GTA, Witcher, Red_dead, SSF).interpretation(Game)

parser = Parser(GAME) 

In [0]:
matches = []

for sent in data.text[:200]:
    for match in parser.findall(sent):
        matches.append(match.fact)

In [50]:
for m in matches:
    print(m.name, m.version)

фифа 15
гта 5
гта 4
видмак 3
FIFA 16
GTA 5
Fifa 15
gta 5
GTA V
Super Street Fighter 5
GTA 5
GTA V
GTA V
GTA 5
гта 5
GTA V
FIFA 16
FIFA 16
GTA 5
FIFA 16
FIFA 15
GTA V
GTA 4
FIFA 17
gta 5
Fifa 15


In [53]:
Game = fact(
    'Game',
    ['name', 'subname', 'version']
)

Battlefield = rule(
    morph_pipeline(['Battlefield', 'Батлфилд', "батлфилд", 'battlefield']).interpretation(Game.name), 
    morph_pipeline(['Bad Company', '1943', 'Heroes', 'Hardline']).interpretation(Game.subname),
    morph_pipeline(['I', '1', 'II', '2', 'III', '3', 'IV', '4', 'V', '5']).interpretation(Game.version).optional())

Duty = rule(
    morph_pipeline(['Call Of Duty', 'Call of Duty', "сал оф дьюти", 'Сал Оф Дьюти']).interpretation(Game.name), 
    morph_pipeline(['Black Ops', 'Modern Warfare', 'WWII', 'World at War', 'Модерн Варфаре', 'модерн варфаре', 'варфаре', 'Варфаре']).interpretation(Game.subname),
    morph_pipeline(['I', '1', 'II', '2', 'III', '3', 'IV', '4', 'V', '5']).interpretation(Game.version).optional())
    
Game = or_(Battlefield, Duty).interpretation(GAME)

parser = Parser(GAME) 

TypeError: ignored